In [1]:
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam, RMSprop
from torch.optim.lr_scheduler import OneCycleLR, StepLR, LinearLR
from data.preprocessing import *
from data.data_utils import *
from models.mult2multatt import *
from utils import *

In [34]:
hidden_size_fc = 100
hidden_size_mg = 100
architecture = (100,50,25,5)
rnn_num_layers = 5
model_num_layers = 1
sequence_length_hour = 96  #last 5 days
sequence_length_minute = 300 #minute
dict_values = ['dst_kyoto', 'kp_gfz']
time_steps = 0 #after how time steps you want to inference

In [26]:
device = get_default_device()

In [27]:
device

device(type='cuda')

In [35]:
arch = 'LSTM'
encoder_fc = ResidualMultiheadAttentionLSTM(input_size = 9,hidden_size =  hidden_size_fc, num_heads=[3,10], num_layers_lstm = rnn_num_layers, bidirectional=False, overall_num_layers=model_num_layers)
encoder_mg = ResidualMultiheadAttentionLSTM(input_size = 11, hidden_size = hidden_size_mg, num_heads=[11, 10], num_layers_lstm = rnn_num_layers, bidirectional=False, overall_num_layers=model_num_layers)
decoder = ResidualMultiheadAttentionLSTM(input_size = encoder_mg.hidden_size+encoder_fc.hidden_size, hidden_size = hidden_size_fc+hidden_size_mg, num_heads = [20,20], num_layers_lstm = rnn_num_layers, bidirectional=False, overall_num_layers=model_num_layers)
model = to_device(MultiHeaded2MultiheadAttentionLSTM(encoder_fc, encoder_mg, decoder, [10,10], architecture, task = 'regression'), device)

In [6]:
#DATA PROCESSING
start_time = '20210101'
end_time = '20230802'
scrap_date = interval_time(start_time, end_time)
months = list(set([day[:6] for day in scrap_date]))
import_Dst(months)
l1_sample, l2_sample, dst, kp = automated_preprocessing(scrap_date, sep = True)
l1_sample_hour = (l1_sample[0].resample('60min').mean(), l1_sample[1].resample('60min').mean()) #multhead
l2_sample_hour = (l2_sample[0].resample('60min').mean(), l2_sample[1].resample('60min').mean()) #multhead encoder forcing

c:\Users\jorge\Desktop\SMFGF\SMFGF-SpaceApps\data\preprocessing.py:360: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  today_kp = kp[day][0:8]


In [29]:
hour_kp_dataset = MainToSingleTarget(l1_sample_hour, dst, sequence_length_hour, 1, hour = True, sep = True, target_mode = 'dst_kyoto', l2_df = l2_sample_hour, time_step_ahead = 0, dae = False, multiclass = False)

In [30]:
#Test:5% training: 90%

test_size = round(0.05*len(hour_kp_dataset))

train_hour_kp, test_hour_kp = random_split(hour_kp_dataset , [len(hour_kp_dataset) - test_size, test_size])

batch_size = 100  #Change based on GPU capacity

train_hour_kp_dl = DataLoader(train_hour_kp, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_hour_kp_dl = DeviceDataLoader(train_hour_kp_dl, device)
test_hour_kp_dl = DataLoader(test_hour_kp, batch_size*2, num_workers=4, pin_memory=True)
test_hour_kp_dl = DeviceDataLoader(test_hour_kp_dl, device)

In [31]:
history = []

In [41]:
##hyperparameters
epochs = 100
max_lr =1e-2
weigth_decay = 1e-6
grad_clip = 1e-2
opt_func = Adam
#lr_sched = OneCycleLR  
lr_sched = OneCycleLR
start_factor = 1
end_factor = 0.1
steps = epochs
gamma = 0.999
weights = [0.1,0.1,0.2,1]
encoder_forcing = True
#opt_func = RMSprop

In [42]:
history += model.fit(epochs, max_lr, train_hour_kp_dl, test_hour_kp_dl, weigth_decay, grad_clip, opt_func, lr_sched, start_factor, end_factor, steps, gamma, weights, encoder_forcing)

Epoch [0]:
	last_lr: 0.00043
	train_loss: 1.0098
	val_loss: 0.8473
	r2_score: -0.0079
Epoch [1]:
	last_lr: 0.00050
	train_loss: 1.0096
	val_loss: 0.8471
	r2_score: -0.0078
Epoch [2]:
	last_lr: 0.00063
	train_loss: 1.0100
	val_loss: 0.8473
	r2_score: -0.0078
Epoch [3]:
	last_lr: 0.00081
	train_loss: 1.0103
	val_loss: 0.8473
	r2_score: -0.0078
Epoch [4]:
	last_lr: 0.00104
	train_loss: 1.0089
	val_loss: 0.8472
	r2_score: -0.0078
Epoch [5]:
	last_lr: 0.00132
	train_loss: 1.0089
	val_loss: 0.8474
	r2_score: -0.0080
Epoch [6]:
	last_lr: 0.00163
	train_loss: 1.0095
	val_loss: 0.8475
	r2_score: -0.0080
Epoch [7]:
	last_lr: 0.00199
	train_loss: 1.0110
	val_loss: 0.8470
	r2_score: -0.0078
Epoch [8]:
	last_lr: 0.00238
	train_loss: 1.0091
	val_loss: 0.8475
	r2_score: -0.0080
Epoch [9]:
	last_lr: 0.00280
	train_loss: 1.0098
	val_loss: 0.8471
	r2_score: -0.0078
Epoch [10]:
	last_lr: 0.00325
	train_loss: 1.0082
	val_loss: 0.8474
	r2_score: -0.0079
Epoch [11]:
	last_lr: 0.00372
	train_loss: 1.0093
	va

KeyboardInterrupt: 

In [ ]:
import os
os.makedirs('mult2mult', exist_ok=True)
torch.save(model, f'mult2mult/{hidden_size_fc}_{hidden_size_mg}_{rnn_num_layers}_{encoder_forcing}_{time_steps}_{arch}.pt')
try:     
    os.remove(f'mult2mult/{hidden_size_fc}_{hidden_size_mg}_{rnn_num_layers}_{encoder_forcing}_{time_steps}_{arch}.csv')
except FileNotFoundError:   
    pass
with open(f'mult2mult/{hidden_size_fc}_{hidden_size_mg}_{rnn_num_layers}_{encoder_forcing}_{time_steps}_{arch}.csv', 'w') as file:
    file.write(','.join(map(str,list(history[-1].values()))))

IndexError: list index out of range